In [1]:
# 导入库
import torch
import torch.nn as nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

In [30]:
# 确定数据 确定超参数
lr = 0.15 # 学习率
gamma = 0 # 动量
epochs = 10 # 全部数据学习次数
bs = 128   # 小批量梯度数据个数

In [3]:
# 导入数据
mnist = torchvision.datasets.FashionMNIST(root="./data/"
                                          , download=False
                                          , train=True
                                          , transform=transforms.ToTensor()
                                         )

In [4]:
mnist

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: ./data/
    Split: Train
    StandardTransform
Transform: ToTensor()

In [5]:
# 小批量划分
batchdata = DataLoader(mnist, batch_size=bs, shuffle=True)

In [6]:
# 查看结构
for x, y in batchdata:
    print(x.shape)
    print(y.shape)
    break

torch.Size([128, 1, 28, 28])
torch.Size([128])


In [9]:
# 对于torch.Size([128, 1, 28, 28])四维 通常将后面的元素乘积作为神经元的输入
input_ = mnist.data[0].numel()
input_

784

In [10]:
# 标签的长度
output_ = len(mnist.targets.unique())
output_

10

In [12]:
# 定义神经网络
class Model(nn.Module):
    def __init__(self, in_features=10, out_features=10):
        super().__init__()
        self.linear1 = nn.Linear(in_features, 128, bias=False)
        self.output = nn.Linear(128, out_features, bias=False)
        
    def forward(self, x):
        x = x.view(-1, 28 * 28) # -1占位符
        sigma1 = torch.relu(self.linear1(x))
        sigma2 = F.log_softmax(self.output(sigma1), dim=1)
        return sigma2

In [27]:
# 定义训练函数 损失函数 优化算法 梯度下降
def fit(net, batchdata, lr=0.1, epochs=10, gamma=0):
    criterion = nn.NLLLoss()
    opt = optim.SGD(net.parameters(), lr=lr, momentum=gamma)
    count = 0
    correct = 0
    for epoch in range(epochs):
        for batch_idx, (x, y) in enumerate(batchdata):
            y = y.view(x.shape[0])
            sigma = net.forward(x) # 正向传播
            loss = criterion(sigma, y)
            loss.backward() # 反向传播
            opt.step() # 梯度下降
            opt.zero_grad() # 梯度清零
            
            yhat = torch.max(sigma, 1)[1] # 取出索引
            correct += torch.sum(yhat == y)
            count += x.shape[0]
            if (batch_idx + 1) % 125 == 0: # 隔一段时间打印一次
                print("Epoch{}:[{}/{}({:.0f}%)],Loss:{:.6f},Accuracy:{:.3f}".format(
                                                      epoch+1, 
                                                      count, 
                                                      epochs * len(batchdata.dataset), 
                                                      100 * count / (epochs * len(batchdata.dataset)),
                                                      loss.data.item(),
                                                      float (100 * correct / count)))

In [31]:
# 训练
torch.manual_seed(428)
net = Model(in_features=input_, out_features=output_)
fit(net, batchdata, lr=lr, epochs=epochs, gamma=gamma)

Epoch1:[16000/600000(3%)],Loss:0.680651,Accuracy:65.194
Epoch1:[32000/600000(5%)],Loss:0.516218,Accuracy:70.863
Epoch1:[48000/600000(8%)],Loss:0.620159,Accuracy:73.852
Epoch2:[76000/600000(13%)],Loss:0.473786,Accuracy:76.818
Epoch2:[92000/600000(15%)],Loss:0.530770,Accuracy:77.935
Epoch2:[108000/600000(18%)],Loss:0.387089,Accuracy:78.838
Epoch3:[136000/600000(23%)],Loss:0.378450,Accuracy:79.973
Epoch3:[152000/600000(25%)],Loss:0.363687,Accuracy:80.459
Epoch3:[168000/600000(28%)],Loss:0.354691,Accuracy:80.961
Epoch4:[196000/600000(33%)],Loss:0.295524,Accuracy:81.631
Epoch4:[212000/600000(35%)],Loss:0.370026,Accuracy:81.938
Epoch4:[228000/600000(38%)],Loss:0.344174,Accuracy:82.227
Epoch5:[256000/600000(43%)],Loss:0.284875,Accuracy:82.674
Epoch5:[272000/600000(45%)],Loss:0.429177,Accuracy:82.917
Epoch5:[288000/600000(48%)],Loss:0.363779,Accuracy:83.125
Epoch6:[316000/600000(53%)],Loss:0.406582,Accuracy:83.497
Epoch6:[332000/600000(55%)],Loss:0.410082,Accuracy:83.687
Epoch6:[348000/600000(